##  Daily Stock Price ETL Pipeline

# STEP 1: Import Libraries

In [3]:
!pip install yfinance

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import os

# Create output directory if not exists
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# STEP 2: Define Parameters

In [6]:
TICKERS = ['AAPL', 'MSFT', 'TSLA']
PERIOD = '3mo'
DB_PATH = 'sqlite:///finance_etl.db'
TABLE_NAME = 'stock_prices'

# STEP 3: Extract Data

In [9]:
def extract_data(tickers, period):
    print(f"[{datetime.now()}] Starting data extraction...")
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, period=period, auto_adjust=True)
        data[ticker] = df
        print(f"Extracted data for {ticker}")
    return data

raw_data = extract_data(TICKERS, PERIOD)

[2025-05-19 23:32:23.762271] Starting data extraction...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Extracted data for AAPL


[*********************100%***********************]  1 of 1 completed

Extracted data for MSFT
Extracted data for TSLA


# STEP 4: Transform Data

In [11]:
def transform_data(data_dict):
    print(f"[{datetime.now()}] Transforming data...")
    all_transformed = []

    for ticker, df in data_dict.items():
        df = df.copy()
        df['Ticker'] = ticker
        df['Daily_Return'] = df['Close'].pct_change()
        df['MA_20'] = df['Close'].rolling(window=20).mean()
        df['MA_50'] = df['Close'].rolling(window=50).mean()
        df.dropna(inplace=True)
        all_transformed.append(df)

    final_df = pd.concat(all_transformed)
    final_df.reset_index(inplace=True)
    return final_df

transformed_data = transform_data(raw_data)

[2025-05-19 23:33:48.888658] Transforming data...


# STEP 5: Load to Database

In [12]:
def load_to_db(df, db_path, table_name):
    print(f"[{datetime.now()}] Loading data to database...")
    engine = create_engine(db_path)
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print("Data loaded successfully.")

load_to_db(transformed_data, DB_PATH, TABLE_NAME)

[2025-05-19 23:34:04.293834] Loading data to database...
Data loaded successfully.


# STEP 6: Visualize Trends & Save Plots

In [17]:
# Plot Close Price with Moving Averages and Save
def plot_price_ma(ticker, df):
    df_ticker = df[df['Ticker'] == ticker]
    plt.figure(figsize=(12, 6))
    plt.plot(df_ticker['Date'], df_ticker['Close'], label='Close')
    plt.plot(df_ticker['Date'], df_ticker['MA_20'], label='MA 20')
    plt.plot(df_ticker['Date'], df_ticker['MA_50'], label='MA 50')
    plt.title(f"{ticker} Price with Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.tight_layout()
    
    plot_path = os.path.join(OUTPUT_DIR, f"{ticker}_price_ma.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"Saved: {plot_path}")

for ticker in TICKERS:
    plot_price_ma(ticker, transformed_data)

# Plot Daily Returns and Save
plt.figure(figsize=(10, 5))
for ticker in TICKERS:
    df_ticker = transformed_data[transformed_data['Ticker'] == ticker]
    plt.plot(df_ticker['Date'], df_ticker['Daily_Return'], label=ticker)
plt.title("Daily Returns Over Time")
plt.xlabel("Date")
plt.ylabel("Return")
plt.legend()
plt.tight_layout()

returns_path = os.path.join(OUTPUT_DIR, "daily_returns.png")
plt.savefig(returns_path)
plt.close()
print(f"Saved: {returns_path}")

Saved: outputs\AAPL_price_ma.png
Saved: outputs\MSFT_price_ma.png
Saved: outputs\TSLA_price_ma.png
Saved: outputs\daily_returns.png


# Save Final Transformed Data to CSV

In [18]:
csv_path = os.path.join(OUTPUT_DIR, "transformed_stock_data.csv")
transformed_data.to_csv(csv_path, index=False)
print(f"Transformed data saved to: {csv_path}")

Transformed data saved to: outputs\transformed_stock_data.csv
